In [36]:
# Стандартизация
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('data/occupancy_even.csv')

scaler = preprocessing.StandardScaler()

names = df.columns
scaled_data = scaler.fit_transform(df)

df_std = pd.DataFrame(scaled_data, columns=names)


In [37]:
# Создание и обучение классификатора
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

x = df_std.iloc[:, :-1].values  # все колонки до последней - переменные функции
y = df.iloc[:, -1].values  # в качестве результирующего значения берем последнюю колонку

# делим набор данных на обучающий и тестовый
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

classifier = GaussianNB()
classifier.fit(x_train, y_train)  # идентифицирует и изучает параметры модели из обучающего набора данных


GaussianNB()

In [38]:
# Оценка качества классификатора
y_pred = classifier.predict(x_test)  # прогноз на основе тестовых данных

# классификационная оценка точности
print("Точность:\n", accuracy_score(y_test, y_pred), "\n")

# вычисляет матрицу ошибок для оценки точности классификации
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred), "\n")


Точность:
 0.9701789264413518 

Матрица ошибок:
 [[261  13]
 [  2 227]] 



In [39]:
# Генератор входных данных
import pandas as pd
import numpy as np

n = 15  # количество входных объектов

# генерируем параметры в диапазоне, большем допустимого интервала
temp = np.random.uniform(low=15.0, high=30.0, size=(n,))
humid = np.random.uniform(low=10.0, high=40.0, size=(n,))
light = np.random.uniform(low=0.0, high=1200.0, size=(n,))
co2 = np.random.uniform(low=300.0, high=2000.0, size=(n,))
humid_ratio = np.random.uniform(low=0.001, high=0.007, size=(n,))
occupancy = np.zeros((n,))

matrix_aux = np.vstack([temp, humid, light, co2, humid_ratio, occupancy])
matrix = np.transpose(matrix_aux)
df_input = pd.DataFrame(matrix, columns=['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy'])

print(df_input)


    Temperature   Humidity        Light          CO2  HumidityRatio  Occupancy
0     17.066686  26.863032    51.744275  1389.121874       0.004455        0.0
1     21.595031  21.657614   729.308258  1051.735349       0.004239        0.0
2     23.947859  29.200725  1006.834511  1998.767881       0.006298        0.0
3     20.705933  20.368108   185.528205  1619.208862       0.003014        0.0
4     20.693858  23.368744  1132.833617  1185.997764       0.003869        0.0
5     26.083343  12.656207   844.812449  1470.564385       0.003570        0.0
6     17.342311  31.975043   282.702372   841.087580       0.004290        0.0
7     29.403042  28.646465   994.169116  1929.827852       0.005461        0.0
8     16.912578  32.307598   992.388053   971.805506       0.003776        0.0
9     19.366455  30.404387   693.346533   890.971757       0.006091        0.0
10    23.245324  18.437824   421.241934   759.390168       0.004927        0.0
11    20.745306  23.053320   612.583794  1250.635296

In [40]:
# Стандартизируем тестовые данные
import pandas as pd

input_scaled = scaler.transform(df_input)

df_input_std = pd.DataFrame(input_scaled, columns=names)
print(df_input_std)


    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
0     -4.209173  0.455423 -0.771982  2.016601       0.507900  -0.963873
1     -0.017963 -1.621742  1.884967  0.910486       0.166671  -0.963873
2      2.159698  1.388254  2.973238  4.015313       3.412278  -0.963873
3     -0.840868 -2.136306 -0.247372  2.770937      -1.763741  -0.963873
4     -0.852043 -0.938935  3.467321  1.350663      -0.415870  -0.963873
5      4.136195 -5.213656  2.337896  2.283609      -0.888236  -0.963873
6     -3.954068  2.495316  0.133680  0.219882       0.247069  -0.963873
7      7.208742  1.167083  2.923573  3.789295       2.092628  -0.963873
8     -4.351807  2.628018  2.916588  0.648438      -0.562794  -0.963873
9     -2.080621  1.868563  1.743949  0.383426       3.086334  -0.963873
10     1.509466 -2.906565  0.676938 -0.047961       1.251542  -0.963873
11    -0.804426 -1.064801  1.427252  1.562576       2.792266  -0.963873
12    -0.240660  5.260748  0.105473 -0.260949      -0.804939  -0

In [41]:
# Удаляем значения вне доверительного интервала
std = df_std.std()

for_deletion = df_input_std.index[
    (df_input_std.Temperature < (-3 * std.Temperature)) |
    (df_input_std.Temperature > (3 * std.Temperature)) |
    (df_input_std.Humidity < (-3 * std.Humidity)) |
    (df_input_std.Humidity > (3 * std.Humidity)) |
    (df_input_std.Light < (-3 * std.Light)) |
    (df_input_std.Light > (3 * std.Light)) |
    (df_input_std.CO2 < (-3 * std.CO2)) |
    (df_input_std.CO2 > (3 * std.CO2)) |
    (df_input_std.HumidityRatio < (-3 * std.HumidityRatio)) |
    (df_input_std.HumidityRatio > (3 * std.HumidityRatio))
    ].tolist()

df_input_clean = df_input.drop(for_deletion)
df_input_clean_std = df_input_std.drop(for_deletion)

print(df_input_clean, '\n')
print(df_input_clean_std)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy
1     21.595031  21.657614  729.308258  1051.735349       0.004239        0.0
3     20.705933  20.368108  185.528205  1619.208862       0.003014        0.0
10    23.245324  18.437824  421.241934   759.390168       0.004927        0.0
11    20.745306  23.053320  612.583794  1250.635296       0.005904        0.0 

    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
1     -0.017963 -1.621742  1.884967  0.910486       0.166671  -0.963873
3     -0.840868 -2.136306 -0.247372  2.770937      -1.763741  -0.963873
10     1.509466 -2.906565  0.676938 -0.047961       1.251542  -0.963873
11    -0.804426 -1.064801  1.427252  1.562576       2.792266  -0.963873


In [42]:
# Подаем полученные данные на вход классификатору

input_x = df_input_clean_std.iloc[:, :-1].values

input_y = classifier.predict(input_x)
probs = classifier.predict_proba(input_x)
probs = np.transpose(probs)

df_input_clean.Occupancy = input_y
df_input_clean['Probability0'] = probs[0]
df_input_clean['Probability1'] = probs[1]

print(df_input_clean)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy  \
1     21.595031  21.657614  729.308258  1051.735349       0.004239          1   
3     20.705933  20.368108  185.528205  1619.208862       0.003014          1   
10    23.245324  18.437824  421.241934   759.390168       0.004927          1   
11    20.745306  23.053320  612.583794  1250.635296       0.005904          1   

    Probability0  Probability1  
1   1.972545e-13       1.00000  
3   3.802445e-04       0.99962  
10  2.214131e-08       1.00000  
11  1.471002e-17       1.00000  
